# 20 뉴스 그룹 분류

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all', random_state=2021)

## 데이터 탐색

In [4]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
print(news.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [7]:
# 타겟 클래스의 값과 분포도
pd.Series(news.target).value_counts().sort_index()

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

In [8]:
print(news.data[0])

From: dagibbs@quantum.qnx.com (David Gibbs)
Subject: Re: Countersteering sans Hands
Organization: QNX Software Systems, Ltd.
Lines: 22

In article <1993Apr20.203344.8417@cs.cornell.edu> karr@cs.cornell.edu (David Karr) writes:
>In article <Clarke.6.735328328@bdrc.bd.com> Clarke@bdrc.bd.com (Richard Clarke) writes:
>>So how do I steer when my hands aren't on the bars? (Open Budweiser in left 
>>hand, Camel cigarette in the right, no feet allowed.) 
>
>>If I lean, and the 
>>bike turns, am I countersteering?
>
>No, the bars would turn only *toward* the direction of turn in
>no-hands steering.

Just in case the original poster was looking for a serious answer,
I'll supply one.

Yes, even when steering no hands you do something quite similar
to countersteering.  Basically to turn left, you to a quick wiggle
of the bike to the right first, causing a counteracting lean to
occur to the left.  It is a lot more difficult to do on a motorcycle
than a bicycle though, because of the extra weight. 

### 훈련/테스트용 데이터 추출

In [36]:
train_news = fetch_20newsgroups(subset='train', random_state=2021,
                                remove=('headers','footers','quotes'))
X_train = train_news.data
y_train = train_news.target

In [38]:
print(X_train[1])
print(y_train[1])

]Is it possible to do a "wheelie" on a motorcycle with shaft-drive?

yes.

8


In [40]:
test_news = fetch_20newsgroups(subset='test', random_state=2021,
                               remove=('headers','footers','quotes'))
X_test = test_news.data
y_test = test_news.target

In [44]:
len(X_train), len(X_test)

(11314, 7532)

## 피처 백터화
- case 1) Counter Vectorizer

In [49]:

from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_count = cvect.transform(X_train)
X_test_count = cvect.transform(X_test)

In [50]:
X_train_count.shape

(11314, 101631)

In [52]:
X_test_count.shape

(7532, 101631)

## 
- case 2) Tfidf Vectorizer

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tfidf = tvect.transform(X_train)
X_test_tfidf = tvect.transform(X_test)

In [68]:
X_train_tfidf.shape, X_test_tfidf.shape

((11314, 101631), (7532, 101631))

### Logistic Regrsssion 분류

In [69]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

- CountVectorizer로 백터화한 데이터

In [71]:
lr.fit(X_train_count, y_train)
pred_count = lr.predict(X_test_count)

C:\Users\BSH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_count)

0.6043547530536378

- TfidfVectorizer로 벡터화한 데이터

In [77]:
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)
pred_tfidf = lr.predict(X_test_tfidf)
accuracy_score(y_test, pred_tfidf)

0.6736590546999469

### Decision Tree로 분류

In [78]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train_tfidf, y_train)
pred_dtc = dtc.predict(X_test_tfidf)
accuracy_score(y_test, pred_dtc)

0.4028146574614976

### Naive Bayes로 분류

In [80]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
pred_nb = nb.predict(X_test_tfidf)
accuracy_score(y_test, pred_nb)

0.6062134891131173

- Case 3) Tfidf, ngram_range(1,2), stopwords, max_df=300

In [81]:
tvect3 = TfidfVectorizer(ngram_range=(1,2), stop_words='english',max_df=300)
tvect3.fit(X_train)
X_train_tfidf3 = tvect3.transform(X_train)
X_test_tfidf3 = tvect3.transform(X_test)

In [86]:
lr = LogisticRegression()
lr.fit(X_train_tfidf3, y_train)
pred3 = lr.predict(X_test_tfidf3)
accuracy_score(y_test, pred3)

0.6922464152947424

In [85]:
X_train_tfidf.shape, X_train_tfidf3.shape

((11314, 101631), (11314, 943453))

- Case 4) Case 3에서 Logistic Regression 파라미터 c의 값을 10으로

In [87]:
%time
lr4 = LogisticRegression(C=10)
lr4.fit(X_train_tfidf3, y_train)
pred4 = lr.predict(X_train_tfidf3)
accuracy_score(y_test, pred4)

Wall time: 0 ns
C:\Users\BSH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Found input variables with inconsistent numbers of samples: [7532, 11314]

In [ ]:
%time lr_run()